In [1]:
%gui qt
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
from src.define.path import contrast
from src.data.io import read_serie, show_image
from src.process.range import window_image, normalize
from src.methods.aorta_contour import by_point

In [17]:
import networkx as nx
from scipy.stats import norm
import cv2
from src.process import contour

def outer_wall(image, aorta, expand_coef=1, width=10):
    def expand(aorta, alpha):
        contours, _ = cv2.findContours(aorta.astype("uint8"), cv2.RETR_TREE, 1)
        aorta_contour = contours[0]
        area = contour.area(aorta_contour)

        ksize = int(alpha * area ** 0.5)
        kernel = np.ones((ksize, ksize), 'uint8')
        return cv2.dilate(aorta.astype('uint8'), kernel).astype('bool')
        
    def gcpdf(image, aorta, initial):
        def get_norm_dist_pdf(mean, var):
            std = var ** 0.5
            def pdf(x):
                return norm.pdf((x - mean) / std) / std
            return pdf

        outer_init = initial.copy()
        outer_init[aorta] = False
        mean = image[outer_init].mean()
        var = image[outer_init].var()
        pdf = get_norm_dist_pdf(mean, var)
        g = nx.Graph()
        g.add_node('s')
        g.add_node('t')
        source_weights = []
        source_min, source_max = float('inf'), float('-inf')
        edge_weights = []
        edge_min, edge_max = float('inf'), float('-inf')
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                if outer_init[i][j]:
                    g.add_node((i, j))
                    dens = pdf(image[i][j])
                    source_weights.append([(i,j), dens])
                    source_min = source_min if source_min < dens else dens
                    source_max = source_max if source_max > dens else dens
                    local_var = float(np.array([[image[i-1][j-1],image[i-1][j],image[i-1][j+1]],
                                          [image[i+0][j-1],image[i+0][j],image[i+0][j+1]],
                                          [image[i+1][j-1],image[i+1][j],image[i+1][j+1]]]).var())
                    edge_min = edge_min if edge_min < local_var else local_var
                    edge_max = edge_max if edge_max > local_var else local_var
                    if outer_init[i+1][j]:
                        edge_weights.append([(i,j), (i+1,j), local_var])
                    if outer_init[i][j+1]:
                        edge_weights.append([(i,j), (i,j+1), local_var])

        source_coef = 100 / (source_max - source_min)
        edge_coef = 100 / (edge_max - edge_min)
                    
        for i in range(len(source_weights)):
            source_weights[i][1] = source_coef * (source_weights[i][1] - source_min)
        for i in range(len(edge_weights)):
            edge_weights[i][2] = edge_coef * (edge_max - edge_weights[i][2])

        m = source_weights[0][1]
        for l in source_weights:
            m = m if m > l[1] else l[1]
        
        g.add_weighted_edges_from([(l[0], 's', l[1]) for l in source_weights])
        g.add_weighted_edges_from([(l[0], 't', 100-l[1]) for l in source_weights])
        g.add_weighted_edges_from(edge_weights)

        _, partition = nx.minimum_cut(g, 's', 't', capacity='weight')
        reachable, unreachable = partition

        unreachable.remove('t')
        reachable.remove('s')
        reach = np.zeros_like(image, dtype='bool')
        for point in reachable:
            reach[point[0], point[1]] = True
        unreach = np.zeros_like(image, dtype='bool')
        for point in unreachable:
            unreach[point[0], point[1]] = True

        return outer_init

    def gcbac(image, initial, width=10):
        current = initial.copy().astype(np.uint8)

        contours, _ = cv2.findContours(current, cv2.RETR_TREE, 1)
        inner_contour = contours[0]
        inner_contour_image = np.zeros_like(image, dtype='uint8')
        cv2.drawContours(inner_contour_image, [inner_contour], -1, 1, width,)

        contours, _ = cv2.findContours(inner_contour_image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        contours.sort(key=len, reverse=True)
        external_contour = contours[0]
        external_contour_image = np.zeros_like(image, dtype='uint8')
        cv2.drawContours(external_contour_image, [external_contour], -1, 1, 1)
        internal_contour = contours[1]
        internal_contour_image = np.zeros_like(image, dtype='uint8')
        cv2.drawContours(internal_contour_image, [internal_contour], -1, 1, 1)

        inner_contour_image = inner_contour_image.astype(np.bool)
        external_contour_image = external_contour_image.astype(np.bool)
        internal_contour_image = internal_contour_image.astype(np.bool)
        inner_contour_image[external_contour_image] = False
        inner_contour_image[internal_contour_image] = False

        g = nx.Graph()
        g.add_node('s')
        g.add_node('t')
        inner_posses = list(zip(*np.where(inner_contour_image==True)))
        g.add_nodes_from(inner_posses)
        external_posses = list(zip(*np.where(external_contour_image)))
        internal_posses = list(zip(*np.where(internal_contour_image)))
        g.add_nodes_from(external_posses)
        g.add_nodes_from(internal_posses)
        g.add_edges_from([(pos, 's') for pos in internal_posses])
        g.add_edges_from([(pos, 't') for pos in external_posses if pos not in internal_posses])

        for i, j in inner_posses:
            g.add_edge((i, j), (i, j+1), capacity=abs(image[i, j] - image[i, j+1]))
            g.add_edge((i, j), (i, j-1), capacity=abs(image[i, j] - image[i, j-1]))
            g.add_edge((i, j), (i+1, j), capacity=abs(image[i, j] - image[i+1, j]))
            g.add_edge((i, j), (i-1, j), capacity=abs(image[i, j] - image[i-1, j]))    

        _, partition = nx.minimum_cut(g, 's', 't', capacity='capacity')
        reachable, unreachable = partition
        reachable.remove('s')
        unreachable.remove('t')

        reachable_canvas = np.zeros_like(image)
        unreachable_canvas = np.zeros_like(image)
        x, y = zip(*reachable)
        reachable_canvas[x, y] = True
        contours, _ = cv2.findContours(reachable_canvas.astype("uint8"), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        contour = contours[0]
        for cont in contours:
            contour = contour if len(contour) >= len(cont) else cont

        result = np.zeros_like(image, dtype=np.uint8)
        cv2.fillConvexPoly(result, contour, 1, 1)
        
        return result.astype('bool')

    
    tuned = None
    current = expand(aorta.copy(), expand_coef)
    for i in range(20):
        segmented = gcpdf(image, aorta, current)
        tuned = gcbac(image, segmented, width)
        current = tuned
    return tuned

In [13]:
def get_picture(pic):
    picture = imageio.imread(Path("unmarked") / pic)
    return picture

def get_center(pic):
    x, y = Path("marks", "centers", str(pic)+".center").open().read().split(", ")
    x, y = int(x), int(y)
    return (x, y)

def get_mask(pic):
    f = Path("marks", "masks", str(pic)+".binary")
    return imageio.imread(f).astype("bool")

In [29]:
dataset = [Path(*f.parts[-2:]) for f in Path("unmarked").glob("**/*.png") if f.parts[-2] != "02" and f.parts[-2] != "01"]
del dataset[-2]

In [15]:
from sklearn.metrics import jaccard_score
import imageio
from sklearn.utils.estimator_checks import check_estimator

class Estimator:
    def __init__(self, expand=1, width=10):
        self.expand = expand
        self.width = width

    def fit(self, X, y=None):
        pass
    
    def predict(self, X, y=None):
        results = []
        for pic in X:
            center = get_center(pic)
            picture = get_picture(pic)
            aorta = by_point(picture, center)
            outer = outer_wall(picture, aorta, self.expand, self.width)
            results.append(outer)
        return results

    def score(self, data):
        labels = self.predict(data)
        scores = 0
        for pic, lab in zip(data, labels):
            truth = get_mask(pic)
            scores += jaccard_score(lab.flatten(), truth.flatten())
        scores /= len(data)
        return scores

    def get_params(self, deep=True):
        return {'expand': self.expand, 'width': self.width}

    def set_params(self, **params):
        for parameter, value in params.items():
            setattr(self, parameter, value)
        return self

In [31]:
from sklearn.model_selection import GridSearchCV

params = {'expand': [0.25, 0.5, 0.75, 1, 1.25], 'width': [2, 5, 10, 15, 20]}
searcher = GridSearchCV(Estimator(), params, cv=2)

searcher.fit(dataset)

IndexError: list index out of range

In [23]:
searcher.best_score_

0.5971603520409732

In [27]:
def show_aorta(ds):
    _, axes = plt.subplots(len(ds), 1, figsize=(6, len(ds) * 6))
    for i, pic in enumerate(ds):
        picture = get_picture(pic)
        center = get_center(pic)
        axes[i].imshow(by_point(picture, center))